# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .

In [ ]:
!pip install datasets

In [ ]:
!pip install evaluate

In [3]:
!mkdir models/

## Download files

In [ ]:
!wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
!wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py

In [ ]:
!python run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file train.txt \
    --validation_file valid.txt \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --block_size 2048 \
    --dataset_config_name plain_text \
    --do_train \
    --do_eval \
    --output_dir models/essays

## Evaluate model

In [17]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [19]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [20]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [21]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
model.cuda()

In [23]:
text = "<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

In [25]:
tok.decode(out[0])

'<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение:  В данном высказывании автор хотел сказать о том значении слова семья для общества в целом или отдельно от него.</S_T] - С одной стороны это выражение подчеркивает связь семьи с понятием воспитания; она выступает как важная составляющая формирования личности ребенка- гражданина России – члена общности людей по признаку принадлежности к той либо иной социальной группе(например родители воспитывают своих детей), а также дает представление об обществе на примерах из истории становления государства Российского до 1917 года.(Аплодисменты). Семья является важнейшим институтом социализации подрастающего поколения во всех сферах общественной жизни страныи оказывает большое влияние не только личностные характеристики членов данного круга отношений («родитель — ребенок»),но имеет важное социальное значение.[1173].Семья играет важную роль при осуществлении государством политики социального кон